In [ ]:
!pip install vaderSentiment nltk
!pip install textblob
!python -m textblob.download_corpora
!pip install flair
!pip install senticnet
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.5 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
 

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from flair.models import TextClassifier
from flair.data import Sentence
from senticnet.senticnet import SenticNet
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
import re
from google.colab import drive
import itertools


In [ ]:

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

senticnet = SenticNet()

def get_senticnet_sentiment(text):
    try:
        
        return float(senticnet.polarity_value(text))
    except KeyError:
        
        return 0.0

In [ ]:

vader_analyzer = SentimentIntensityAnalyzer()
flair_lstm = TextClassifier.load('sentiment-fast').to('cuda')  
flair_bert = TextClassifier.load('sentiment').to('cuda')  



2025-01-09 15:44:14,224 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-fasttext-rnn/sentiment-en-mix-ft-rnn_v8.pt not found in cache, downloading to /tmp/tmpdm6ajlcn


100%|██████████| 1.16G/1.16G [01:06<00:00, 18.7MB/s]

2025-01-09 15:45:21,095 copying /tmp/tmpdm6ajlcn to cache at /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt


2025-01-09 15:45:26,502 removing temp file /tmp/tmpdm6ajlcn
2025-01-09 15:45:41,019 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpeeqrldho


100%|██████████| 253M/253M [00:15<00:00, 17.7MB/s]

...

2025-01-09 15:45:56,943 removing temp file /tmp/tmpeeqrldho


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+|\#', '', tweet)
    tweet = re.sub(r'[^A-Za-z0-9 ]+', '', tweet)
    tweet = tweet.strip().lower()
    return tweet if tweet else "empty"

In [ ]:
def analyze_sentiment_with_gpu(tweet, weights):
    try:

        if not tweet or tweet.strip() in ["", "empty"]:
            return "Neutral"
        w_vader, w_textblob, w_lstm, w_bert, w_senticnet = weights

        vader_score = vader_analyzer.polarity_scores(tweet)['compound']

        textblob_score = TextBlob(tweet).sentiment.polarity

        lstm_sentence = Sentence(tweet)
        flair_lstm.predict(lstm_sentence)
        lstm_score = lstm_sentence.labels[0].score * (1 if "POSITIVE" in str(lstm_sentence.labels[0]) else -1)

        bert_sentence = Sentence(tweet)
        flair_bert.predict(bert_sentence)
        bert_score = bert_sentence.labels[0].score * (1 if "POSITIVE" in str(bert_sentence.labels[0]) else -1)

        senticnet_score = get_senticnet_sentiment(tweet)

        combined_score = (
            w_vader * vader_score +
            w_textblob * textblob_score +
            w_lstm * lstm_score +
            w_bert * bert_score +
            w_senticnet * senticnet_score
        ) / sum(weights)

        if combined_score < -0.35:
          sentiment_label = "Negative"
        elif combined_score > 0.35:
          sentiment_label = "Positive"
        else:
          sentiment_label = "Neutral"

        return {
        "Combined_Score": combined_score,
        "Sentiment_Label": sentiment_label
        }

    except Exception as e:
        print(f"Error processing tweet: {tweet}\nError: {e}")
        return "Error"

In [ ]:
def evaluate_model(df, weights):
    df['predicted_sentiment'] = df['cleaned_tweet'].apply(lambda x: analyze_sentiment_with_gpu(x, weights))

    df['predicted_sentiment'] = df['predicted_sentiment'].apply(lambda x: x['Sentiment_Label'] if isinstance(x, dict) else x)


    label_map = {"Negative": -1, "Neutral": 0, "Positive": 1}
    df['predicted_sentiment'] = df['predicted_sentiment'].map(label_map)

    y_true = df['sentiment']
    y_pred = df['predicted_sentiment']
    return f1_score(y_true, y_pred, average='weighted')

In [ ]:
def optimize_weights(df, initial_weights, weight_range=(0.2, 0.8), step=0.2):
    """
    Optimizes the weights for sentiment analysis models using grid search.

    Args:
        df: The DataFrame containing the tweets and sentiment labels.
        initial_weights: The initial weights for the models.
        weight_range: The range of values to consider for each weight.
        step: The step size for iterating through the weight range.

    Returns:
        The optimized weights that yield the highest F1 score.
    """

    best_f1 = 0
    best_weights = initial_weights

    weight_values = np.arange(weight_range[0], weight_range[1] + step, step)
    all_weight_combinations = list(itertools.product(weight_values, repeat=len(initial_weights)))

    with tqdm(total=len(all_weight_combinations), desc="Optimizing Weights") as pbar:
        for weights in all_weight_combinations:
            pbar.set_description(f"Optimizing Weights: {weights}")

            current_f1 = evaluate_model(df, weights)
            pbar.write(f"Weights: {weights}, F1 Score = {current_f1}")

            if current_f1 > best_f1:
                best_f1 = current_f1
                best_weights = weights

            pbar.update(1)

    return best_weights

In [ ]:
def load_sentiment140(file_path):
    df = pd.read_csv(file_path, encoding='latin-1', header=None)
    df.columns = ['sentiment', 'id', 'date', 'flag', 'user', 'text']

    df['sentiment'] = df['sentiment'].map({0: -1, 4: 1})  
    df['cleaned_tweet'] = df['text'].apply(clean_tweet)

    negative_samples = df[df['sentiment'] == -1].sample(n=1000, random_state=42)
    positive_samples = df[df['sentiment'] == 1].sample(n=1000, random_state=42)

    sampled_df = pd.concat([negative_samples, positive_samples]).sample(frac=1, random_state=42)

    train_df = sampled_df.sample(frac=0.8, random_state=42)  
    test_df = sampled_df.drop(train_df.index)  
    return train_df, test_df

In [ ]:
def process_file_with_optimized_weights(file_path, output_path, optimized_weights):
    df = pd.read_csv(file_path)

    if 'cleaned_tweet' not in df.columns:
        raise ValueError("The file must contain a 'cleaned_tweet' column.")

    tqdm.pandas(desc="Applying Sentiment Analysis")
    df['sentiment'] = df['cleaned_tweet'].progress_apply(
        lambda x: analyze_sentiment_with_gpu(x, optimized_weights)
    )

    df.to_csv(output_path, index=False)
    print(f"Sentiment analysis completed. Results saved to {output_path}.")


In [ ]:
data_path = "/content/drive/MyDrive/preprocessed_data/sentiment140.csv"
train_df, test_df = load_sentiment140(data_path)
initial_weights = [0.1, 0.1, 0.1, 0.1, 0.1]

print("Checking for empty tweets...")
empty_tweets = train_df[train_df['cleaned_tweet'].str.strip() == ""]
if not empty_tweets.empty:
    print(f"Found {len(empty_tweets)} empty tweets. Investigate further!")


Checking for empty tweets...


In [ ]:
print("Optimizing weights...")
optimized_weights = optimize_weights(train_df, initial_weights, weight_range=(0.2, 0.8), step=0.2)
print(f"Optimized Weights: {optimized_weights}")

test_f1 = evaluate_model(test_df, optimized_weights)
print(f"F1 Score on Test Set: {test_f1}")

your_file_path = "/content/drive/MyDrive/preprocessed_data/predicted_genders_filtered.csv"
output_path = "/content/drive/MyDrive/preprocessed_data/predicted_sentimentOptimized.csv"
process_file_with_optimized_weights(your_file_path, output_path, optimized_weights)

Optimizing weights...


Optimizing Weights: (0.2, 0.2, 0.2, 0.2, 0.4):   0%|          | 1/1024 [00:14<4:14:11, 14.91s/it]

Weights: (0.2, 0.2, 0.2, 0.2, 0.2), F1 Score = 0.6080108988419091


Optimizing Weights: (0.2, 0.2, 0.2, 0.2, 0.6000000000000001):   0%|          | 2/1024 [00:26<3:44:49, 13.20s/it]

Weights: (0.2, 0.2, 0.2, 0.2, 0.4), F1 Score = 0.5120630100119749


...

Optimizing Weights: (0.8, 0.8, 0.8, 0.8, 0.8): 100%|█████████▉| 1023/1024 [3:27:41<00:12, 12.17s/it]               

Weights: (0.8, 0.8, 0.8, 0.8, 0.6000000000000001), F1 Score = 0.6269337337810218


Optimizing Weights: (0.8, 0.8, 0.8, 0.8, 0.8): 100%|██████████| 1024/1024 [3:27:53<00:00, 12.18s/it]


Weights: (0.8, 0.8, 0.8, 0.8, 0.8), F1 Score = 0.6080108988419091
Optimized Weights: (0.2, 0.2, 0.2, 0.8, 0.2)
F1 Score on Test Set: 0.7147118136555133


Applying Sentiment Analysis:  32%|███▏      | 23674/73227 [03:53<07:43, 106.88it/s]

In [ ]:
def optimize_weights_refined(df, initial_weights, weight_range_width=0.2, step=0.1):
    """
    Refined optimization of weights around initial values.

    Args:
        df: The DataFrame containing the tweets and sentiment labels.
        initial_weights: The initial weights (center of search space).
        weight_range_width: The total width of the search range around each weight.
        step: The step size for iterating through the weight range.

    Returns:
        The optimized weights that yield the highest F1 score.
    """

    best_f1 = 0
    best_weights = initial_weights

    weight_ranges = [
        (max(0, w - weight_range_width / 2), min(1, w + weight_range_width / 2))
        for w in initial_weights
    ]

    weight_values = [
        np.arange(r[0], r[1] + step, step) for r in weight_ranges
    ]

    all_weight_combinations = list(itertools.product(*weight_values))

    with tqdm(total=len(all_weight_combinations), desc="Refining Weights") as pbar:
        for weights in all_weight_combinations:
            pbar.set_description(f"Refining Weights: {weights}")

            current_f1 = evaluate_model(df, weights)
            pbar.write(f"Weights: {weights}, F1 Score = {current_f1}")

            if current_f1 > best_f1:
                best_f1 = current_f1
                best_weights = weights

            pbar.update(1)

    return best_weights

In [ ]:
refined_weights = optimize_weights_refined(
    train_df,
    initial_weights = [0.2, 0.2, 0.2, 0.8, 0.2],  
    weight_range_width=0.2,  
    step=0.1
)
print(f"Refined Weights: {refined_weights}")

Refining Weights: (0.1, 0.1, 0.1, 0.7000000000000001, 0.2):   0%|          | 1/768 [00:13<2:51:05, 13.38s/it]

Weights: (0.1, 0.1, 0.1, 0.7000000000000001, 0.1), F1 Score = 0.7081058987255773


Refining Weights: (0.1, 0.1, 0.1, 0.7000000000000001, 0.30000000000000004):   0%|          | 2/768 [00:25<2:41:57, 12.69s/it]

Weights: (0.1, 0.1, 0.1, 0.7000000000000001, 0.2), F1 Score = 0.7061151159119583


Refining Weights: (0.1, 0.1, 0.1, 0.7000000000000001, 0.4):   0%|          | 3/768 [00:37<2:37:34, 12.36s/it]                

...

Weights: (0.30000000000000004, 0.2, 0.2, 0.9, 0.4), F1 Score = 0.7015412746316351


Refining Weights: (0.30000000000000004, 0.2, 0.30000000000000004, 0.7000000000000001, 0.2):  60%|█████▉    | 457/768 [1:30:30<1:00:20, 11.64s/it]

Weights: (0.30000000000000004, 0.2, 0.30000000000000004, 0.7000000000000001, 0.1), F1 Score = 0.6868222998718926


Refining Weights: (0.30000000000000004, 0.2, 0.30000000000000004, 0.7000000000000001, 0.30000000000000004):  60%|█████▉    | 458/768 [1:30:42<1:00:28, 11.71s/it]

Weights: (0.30000000000000004, 0.2, 0.30000000000000004, 0.7000000000000001, 0.2), F1 Score = 0.6859951885661144


In [ ]:
test_f1 = evaluate_model(test_df, weights=[0.2, 0.2, 0.2, 0.8, 0.2])
print(f"F1 Score on Test Set: {test_f1}")

your_file_path = "/content/drive/MyDrive/preprocessed_data/predicted_genders_filtered.csv"
output_path = "/content/drive/MyDrive/preprocessed_data/predicted_sentimentOptimizedAndRefined.csv"
process_file_with_optimized_weights(your_file_path, output_path, optimized_weights=[0.2, 0.2, 0.2, 0.8, 0.2])

F1 Score on Test Set: 0.7147118136555133


Applying Sentiment Analysis: 100%|██████████| 73227/73227 [11:06<00:00, 109.90it/s]


Sentiment analysis completed. Results saved to /content/drive/MyDrive/preprocessed_data/predicted_sentimentOptimizedAndRefined.csv.
